# Visualising the traffic around Boston Airport

Aim of this notebook is to provide a codealong to be able to recreate the traffic from Square One Mall in Saugus, Massachusetts to Boston Logan Airport.

In [1]:
# Importing the necessary packages:
import pandas as pd
import numpy as np
import json
import requests

In [2]:
#Retrieving my api keys information to access the Google API.
def get_keys(path):
    with open(path) as f:
        return json.load(f)
    
keys = get_keys("/Users/jjherranzsarrion/.secret/google_blog2_api.json")

api_key = keys['api_key']

In [ ]:
url = 'https://maps.googleapis.com/maps/api/directions/json?'

origin = '323+Broadway+Saugus+MA+01906'
destination = 'Terminal+C+Boston+Logan+International+Airport+Boston+MA+02128'
departure_time = '1566214200' #time in seconds from midnight 1st Jan 1970 (Unix start time) until Monday 19th August at 07:30 AM. 
 
url_params = f"origin={origin}&destination={destination}&departure_time={departure_time}&key={api_key}"

request_url = url + url_params
    
print(request_url)    
    
response = requests.get(request_url)
print(type(response.text))
print(response.text)

In [4]:
response.json().keys()

dict_keys(['geocoded_waypoints', 'routes', 'status'])

In [5]:
route = response.json()['routes']

In [8]:
root_steps = []
for legs in route:
    for steps in legs['legs']:
        for coordinates in steps['steps']:
            root_steps.append(coordinates['start_location'])

#Chosen to only select the start coordinate of each step given that the end coordinate of step 0 
#is the start coordinate of step 1. The last coordinate is added at the end of the loop.

In [9]:
# Unpacking the root_steps list to retrieve only latitude and longitude data. Adding weights to represent different
# traffic congestions at each step of the root.

weighted_coordinates = [(root_step['lat'], root_step['lng'], np.random.randint(1,4) ) for root_step in root_steps]

In [25]:
weighted_coordinates_df = pd.DataFrame(weighted_coordinates, columns=['latitude', 'longitude', 'weight'])

In [31]:
import gmaps
import gmaps.datasets
%matplotlib inline

gmaps.configure(api_key)

In [32]:
locations = weighted_coordinates_df[['latitude', 'longitude']]
weights = weighted_coordinates_df['weight']
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig

Figure(layout=FigureLayout(height='420px'))

In [33]:
heatmap_layer = gmaps.heatmap_layer(locations)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
m = gmaps.Map()
heatmap_layer = gmaps.Heatmap(data=weighted_coordinates)
m.add_layer(heatmap_layer)
m